In [ ]:
ptm <- proc.time()

**Import libraries**

In [ ]:
library(rgrass7)
library(rgdal)
library(raster)
library(cluster)
library(randomForest)

**Set GRASS LOCATION** 

    create it if doesn't exist',EPSG:32633

In [ ]:
loc <- initGRASS("/home/epilib/Envs/env1/grass-7.1.svn", home=tempdir())
execGRASS('g.proj',epsg=as.integer(32633), location='utm_wgs84_33N')
execGRASS('g.mapset', mapset='PERMANENT', location='utm_wgs84_33N') 

**Import Bathy**

    # run only once ...

**Bathy Sub set**

    # Resolution: 20m
    #
    # BOUNDARIES:
    #
    #     North: 7840000
    #     South: 7770000
    #     East: 625000
    #     West: 580000

In [ ]:
execGRASS('g.region', raster='elev', res='20', w='580000',e='625000',s='7770000',n='7840000', flags=c('a','p')) 
execGRASS('r.mapcalc', expression = 'minielev=elev', flags='overwrite') 

**Load *morphoclara* function**

    # Input:
    #    elevation: grid used as input
    #    resolution: (FALSE, Int) manualy set the resolution, FALSE use native dataset resolution
    #    win_neighbors:  int, default:9 - Size in pixel for the neighbors moving window operator.
    #                    (used to compute: average, minimun, maximum elevation)
    #    win_param_scale: int, default:15 - Size in pixel for the param_scale moving window operator.
    #                    (used to compute: profile, cross, minimum, longitudinal curvatures)
    #    curvature_tolerance: float, default: 0.0001 (*)
    #    slope_tolerance: float, default: 0.1 (*)
    #    exponent: float, default: 0.0 (*)
    #    zscale: float, default: 1.0 (*)
    #    Remove: boolean, If TRUE remove intermediate products. default FALSE
    #    
    #    (*) : more info at http://grass.osgeo.org/grass71/manuals/r.param.scale.html
    #
    # Output:
    #    brick raster. Layers: 
    #    'er','xslope','profc','crosc','minic','maxic','longc'

In [ ]:
source('morphoclara.R')

In [ ]:
s <- morphoclara(elevation='minielev', remove=TRUE)

In [ ]:
s

In [ ]:
# we can't operate on the entire set of cells,
# so we use one of the central concepts from statistics: sampling
# sample 10000 random points
s.r <- as.data.frame(sampleRegular(s, 10000))

# clara() function: need to remove NA from training set
s.r <- na.omit(s.r)

ncl <- numeric(8)
for (i in 2:9) ncl[i] <- clara(s.r,stand=TRUE,k=i)$ silinfo $ avg.width
plot(2:10,ncl,type="b")

s.clara <- clara(s.r, stand=TRUE, k=6)
s.r$cluster <- factor(s.clara$clustering)

rf <- randomForest(cluster ~ er + xslope + profc + crosc + minic + maxic
                   + longc, data=s.r, importance=TRUE, ntree=201)

# make predictions from rf model, along all cells of input stack
p <- predict(s, rf, type='response', progress='text')

# variable importance: all contribute about the same... good
#importance(rf)
#varImpPlot(rf)

# customized plot (needs an extra library)
#par(mar=c(0,0,0,0))
#plot(p, maxpixels=50000, axes=FALSE, legend=FALSE, col=brewer.pal('Set1', n=5))

writeRaster(p, "TerClass_sub.asc", overwrite=TRUE)

In [ ]:
plot(p)

In [ ]:
execGRASS('r.in.gdal', input='TerClass_sub.asc', output='TerClass_sub', flags=c('overwrite','o','e'))

In [ ]:
execGRASS('r.info', map='TerClass_sub')

In [ ]:
execGRASS('g.region', raster='TerClass_sub', flags=c('a','p')) 

In [ ]:
execGRASS('r.report', flags=c('i','e'), map='TerClass_sub', units=c('me','h','p'), nsteps=255)

In [ ]:
execGRASS('r.univar', map='TerClass_sub')

In [ ]:
proc.time() - ptm